<a href="https://colab.research.google.com/github/Malinowsk/Optativa-IA/blob/main/Rago_Juan_Ignacio_Eugenio_Romanelli_TP_Reglas_de_asociaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP Reglas de Asociación**

Cargamos la librería que nos permite acceder al dataset

In [ ]:
# soporte para cargar dataset de https://www.openml.org/
!pip install openml
import openml

Accedemos al dataset

In [ ]:
import pandas as pd

# indicamos cual dataset queremos utilizar, en este caso el nro. 24
dataset = openml.datasets.get_dataset(24)

# separamos las información almacenada en el dataset
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format='dataframe',
    target=dataset.default_target_attribute
)

#  concatenamos la información relevante en un único DataFrame 
df = pd.concat([X, y], axis=1)
df

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


**a) Describir el dataset seleccionado.**
>El dataset seleccionado contiene información sobre las caracteristicas fisicas de los hongos, clasificandolos en: venenosos o comestibles. Con un total de 23 caracteristicas para 8124 ejemplares de hongos.

>Este dataset corresponde a registros de hongos extraidos de The Audubon Society Field Guide to North American Mushrooms (1981).

**b) Definir cuál es el objetivo de aplicar la técnica de Reglas de Asociación en el dataset elegido. Explicar qué tipo de patrones se esperan encontrar.** 
> El objetivo es descubrir patrones de atributos que suelen ocurrir juntos. Se espera encontrar patrones en los cuales los hongos que son venenesos posean determinadas caracteristicas y los no venenosos posean otras.

**d) Definir qué pre‐procesamiento se realizará a los datos. Justificar. Aplicarlo.**
> El dataset de entrada que debemos proveerle a la función *apriori* debe tener un formato particular llamado "One-hot encoded", para ello debemos realizar un pre-procesamiento utilizando la funcion get_dummies() provista por Pandas.

> Utilizaremos un soprte con valor de 0.3 ya que tenemos muchas transacciones y muchos items, utilizando valores desoporte altos no obtendriamos muchos itemsets frecuentes.



In [ ]:
from mlxtend.frequent_patterns import apriori

#Formato One-hot
df_dummies = pd.get_dummies(df , columns = df.columns)
#print(df_dummies)

#Itemsets frecuentes y soportes asociado.
frequent_itemsets = apriori(df_dummies, min_support=0.3, use_colnames=True )

#Agragamos una columna que almacena la longitud de cada itemset.
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

In [ ]:
#Seleccionamos los itemsets frecuentes que contengan las clases de hongos venenosos.
frequent_itemsets[ frequent_itemsets['itemsets'].apply(lambda x: 'class_p' in x)]

,support,itemsets,length
26,0.482029,(class_p),1
53,0.405219,"(bruises%3F_f, class_p)",2
91,0.479813,"(class_p, gill-attachment_f)",2
107,0.468242,"(gill-spacing_c, class_p)",2
173,0.482029,"(class_p, veil-type_p)",2
...,...,...,...
2526,0.302314,"(gill-spacing_c, ring-number_o, gill-attachmen...",7
2528,0.308223,"(ring-number_o, gill-attachment_f, veil-color_...",7
2529,0.302314,"(gill-spacing_c, ring-number_o, veil-color_w, ...",7
2668,0.335795,"(gill-spacing_c, ring-number_o, gill-attachmen...",7


**e) Obtener las reglas de asociación utilizando el dataset elegido. Definir parámetros del algoritmo. Justificar.**
>Para generar las reglas de asociacion utilizaremos la funcion *associations_rules* pasando como parámetro los itemsetes frecuentes generados, la metrica a utilizar (en nuestro caso será "confidence"), y el valor minimo de confianza (1).


In [ ]:
from mlxtend.frequent_patterns import association_rules

# utilizamos confianza = 1
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cap-shape_f),(gill-attachment_f),0.387986,0.974151,0.381339,0.982868,1.008949,0.003382,1.508835
1,(cap-shape_f),(veil-type_p),0.387986,1.000000,0.387986,1.000000,1.000000,0.000000,inf
2,(cap-shape_f),(veil-color_w),0.387986,0.975382,0.381832,0.984137,1.008976,0.003397,1.551945
3,(cap-shape_f),(ring-number_o),0.387986,0.921713,0.371738,0.958122,1.039501,0.014126,1.869388
4,(cap-shape_x),(gill-attachment_f),0.450025,0.974151,0.443378,0.985230,1.011373,0.004986,1.750096
...,...,...,...,...,...,...,...,...,...
24402,"(bruises%3F_t, stalk-surface-below-ring_s, vei...","(gill-spacing_c, ring-number_o, gill-attachmen...",0.350566,0.484490,0.316100,0.901685,1.861101,0.146254,5.243469
24403,"(bruises%3F_t, stalk-surface-above-ring_s, sta...","(gill-spacing_c, ring-number_o, gill-attachmen...",0.341704,0.772033,0.316100,0.925072,1.198228,0.052294,3.042476
24404,"(bruises%3F_t, stalk-surface-below-ring_s, vei...","(gill-spacing_c, ring-number_o, gill-attachmen...",0.350566,0.484490,0.316100,0.901685,1.861101,0.146254,5.243469
24405,"(bruises%3F_t, ring-number_o, stalk-surface-be...","(gill-spacing_c, gill-attachment_f, veil-color...",0.337765,0.385032,0.316100,0.935860,2.430603,0.186050,9.587910


**f) Determinar qué post‐procesamiento se realizará sobre las reglas obtenidas. Aplicarlo.**
>Para filtrar las reglas que son de nuestro interés utilizaremos las herramientas que prevee la libreria Pandas.
>En nuestro caso buscamos reglas que en su consecuente tengan class_p, lo cual indica que es un hongo venenoso.


Agregamos una columna con la longitud del antecedente

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

In [ ]:
rules[(rules['consequents'] == {'class_p'})]

>Seleccionamos las reglas que poseen al menos tres items en su antecedente, y un lift minimo de 1.2.

In [ ]:
rules[ (rules['antecedent_len'] >= 2) &
      (rules['lift'] > 1.8) &
      (rules['consequents'] == {'class_p'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
206,"(gill-spacing_c, bruises%3F_f)",(class_p),0.435746,0.482029,0.392418,0.900565,1.868281,0.182376,5.209145,2
238,"(bruises%3F_f, population_v)",(class_p),0.334810,0.482029,0.308223,0.920588,1.909821,0.146834,6.522603,2
1083,"(gill-spacing_c, bruises%3F_f, gill-attachment_f)",(class_p),0.409897,0.482029,0.390202,0.951952,1.974887,0.192620,10.780281,3
1160,"(bruises%3F_f, population_v, gill-attachment_f)",(class_p),0.322994,0.482029,0.308223,0.954268,1.979692,0.152530,11.326309,3
1172,"(gill-spacing_c, bruises%3F_f, veil-type_p)",(class_p),0.435746,0.482029,0.392418,0.900565,1.868281,0.182376,5.209145,3
1183,"(gill-spacing_c, bruises%3F_f, veil-color_w)",(class_p),0.412112,0.482029,0.392418,0.952210,1.975423,0.193768,10.838552,3
1190,"(gill-spacing_c, bruises%3F_f, ring-number_o)",(class_p),0.429345,0.482029,0.387986,0.903670,1.874722,0.181030,5.377037,3
1194,"(gill-spacing_c, bruises%3F_f, population_v)",(class_p),0.322994,0.482029,0.302314,0.935976,1.941743,0.146622,8.090221,3
1261,"(bruises%3F_f, population_v, veil-type_p)",(class_p),0.334810,0.482029,0.308223,0.920588,1.909821,0.146834,6.522603,3
1277,"(bruises%3F_f, population_v, veil-color_w)",(class_p),0.322994,0.482029,0.308223,0.954268,1.979692,0.152530,11.326309,3


**g) Analizar las reglas de asociación obtenidas.**

bbb

**h) Mostrar un ejemplo de cómo los resultados obtenidos pueden ser usados en el futuro.**

lkkkgfg